In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt
import hvplot.pandas
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve
import holoviews as hv
from fiona.crs import from_epsg
from geopy.geocoders import Nominatim
from haversine import haversine, Unit
import leafmap

import warnings
warnings.filterwarnings('ignore')
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
#hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}


In [2]:
pd.set_option('display.max_colwidth', None) # To avoid truncated display of DataFrames 


In [153]:
df = pd.read_excel("positiondump utc .xlsx", index_col = [0])


In [154]:
df["long"] = df["GPSpoint"].apply(lambda x: x.split(",")[0])
df["lat"] = df["GPSpoint"].apply(lambda x: x.split(",")[-1])

In [155]:
#Convert Longs and Lats to floats. 

def cnvrtCoord(val: str) -> float:
    sgn = +1
    if val[0] == 'S' or val[0] == 'W':
        sgn = -1
    return float(val[1:]) * sgn  

df['long'] = [cnvrtCoord(x) for x in df['long'].tolist()]
df['lat'] = [cnvrtCoord(x) for x in df['lat'].tolist()]  

df.drop("GPSpoint", axis=1, inplace=True)
df['DateTimeOfPosition'] = pd.to_datetime(df['DateTimeOfPosition'])


In [156]:
df

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
647751005,2023-06-14 18:15:03,0,52.203256,5.972714
647742914,2023-06-14 17:17:28,0,52.203289,5.972705
647742890,2023-06-14 17:17:23,0,52.203289,5.972705
647742726,2023-06-14 17:16:23,1,52.203792,5.975125
647742547,2023-06-14 17:15:23,1,52.200175,5.979859
...,...,...,...,...
631418653,2023-04-01 09:22:53,1,53.391285,5.282144
631418504,2023-04-01 09:21:53,1,53.389952,5.274539
631418385,2023-04-01 09:20:53,1,53.389564,5.270365


In [157]:
df.sort_values("DateTimeOfPosition", inplace=True)

In [158]:
df.drop_duplicates(subset="DateTimeOfPosition", inplace=True)
df.reset_index(inplace=True)


### Finding indexes where ignitionon toggle happens. Then using that to remove records with additional 0

In [159]:
toggles = []
double_zeros = []
data = {}
for i in range(1, len(df)):
    if df.IgnitionOn.iloc[i] != df.IgnitionOn.iloc[i-1]:
        duration_diff = (df.DateTimeOfPosition.iloc[i] - df.DateTimeOfPosition.iloc[i-1])
        distance = haversine((df.lat.iloc[i], df.long.iloc[i]), (df.lat.iloc[i-1], df.long.iloc[i-1]), Unit.METERS)
        if  df.IgnitionOn.iloc[i] == 0:
            if (i+1) < len(df) and df.IgnitionOn.iloc[i] == df.IgnitionOn.iloc[i+1]:
                double_zeros.append(i+1)
            toggles.append((i, df.DateTimeOfPosition.iloc[i] - df.DateTimeOfPosition.iloc[i-1], df.DateTimeOfPosition.iloc[i]))
            #data["DateTimeOfPosition"] = df.DateTimeOfPosition.iloc[i]
            #data["IgnitionOn"] = df.IgnitionOn.iloc[i]

    

In [160]:
len(toggles)

354

In [161]:
keep = [toggle[0] for toggle in toggles]

In [92]:
#keep

In [162]:
df[0:50]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144
5,631418767,2023-04-01 09:23:53,1,53.391549,5.285710
6,631418902,2023-04-01 09:24:53,1,53.392134,5.283950
7,631419027,2023-04-01 09:25:53,1,53.391772,5.284025
8,631419170,2023-04-01 09:26:53,0,53.392237,5.284695
9,631432987,2023-04-01 11:06:38,1,53.392189,5.284687


In [164]:
to_remove = list()
for val in df[df.IgnitionOn == 0].index.values:
    if val == 0: 
        continue
    if val not in keep:
        to_remove.append(val)

In [165]:
len(to_remove)

246

In [166]:
cleaned_df = df.drop(index=to_remove)

In [167]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144
...,...,...,...,...,...
10620,647742183,2023-06-14 17:13:23,1,52.188936,5.980637
10621,647742362,2023-06-14 17:14:23,1,52.193207,5.978572
10622,647742547,2023-06-14 17:15:23,1,52.200175,5.979859
10623,647742726,2023-06-14 17:16:23,1,52.203792,5.975125


### As some records were removed, the indexes were not in consecutive order, there4 has to reset indexes

In [168]:
cleaned_df.reset_index(inplace=True)
cleaned_df.drop("index", axis=1, inplace=True)

In [169]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144
...,...,...,...,...,...
10376,647742183,2023-06-14 17:13:23,1,52.188936,5.980637
10377,647742362,2023-06-14 17:14:23,1,52.193207,5.978572
10378,647742547,2023-06-14 17:15:23,1,52.200175,5.979859
10379,647742726,2023-06-14 17:16:23,1,52.203792,5.975125


In [170]:

# Iterate through the DataFrame rows
for i in range(1, len(cleaned_df.index)):
    # Get the coordinates of the current and previous points
    lon1, lat1 = cleaned_df["long"].iloc[i], cleaned_df["lat"].iloc[i]
    lon2, lat2 = cleaned_df["long"].iloc[i-1], cleaned_df["lat"].iloc[i-1]
    
    # Calculate the Haversine distance between the points
    dist = haversine((lon1, lat1), (lon2, lat2), unit=Unit.KILOMETERS)
    duration = cleaned_df["DateTimeOfPosition"].iloc[i] - cleaned_df["DateTimeOfPosition"].iloc[i-1]
    # Update the Haversine distance in the DataFrame
    cleaned_df.at[i, 'haversine_distance'] = dist
    cleaned_df.at[i, 'duration'] = duration

In [171]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153,NaN,NaT
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988,0.012223,0 days 00:29:24
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365,0.040803,0 days 00:00:59
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539,0.280134,0 days 00:01:00
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144,0.525633,0 days 00:01:00
...,...,...,...,...,...,...,...
10376,647742183,2023-06-14 17:13:23,1,52.188936,5.980637,0.764913,0 days 00:01:00
10377,647742362,2023-06-14 17:14:23,1,52.193207,5.978572,0.495336,0 days 00:01:00
10378,647742547,2023-06-14 17:15:23,1,52.200175,5.979859,0.779757,0 days 00:01:00
10379,647742726,2023-06-14 17:16:23,1,52.203792,5.975125,0.515598,0 days 00:01:00


In [172]:
cleaned_df.sort_values("haversine_distance", ascending=False)

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration
173,631817254,2023-04-03 17:15:39,1,53.177227,5.412910,23.863617,0 days 00:00:59
8228,644338387,2023-05-31 15:42:56,1,52.324896,5.006019,4.263616,0 days 00:01:00
4802,639463194,2023-05-09 13:32:37,1,52.324459,5.005575,4.192140,0 days 00:01:00
6395,641542168,2023-05-19 06:38:12,1,52.187022,5.868476,4.179075,0 days 00:02:00
6400,641542822,2023-05-19 06:44:12,1,52.191777,5.701592,3.985609,0 days 00:02:00
...,...,...,...,...,...,...,...
6826,642426210,2023-05-23 10:36:42,1,51.665755,5.054182,0.000000,0 days 00:01:00
9601,646637230,2023-06-09 17:37:30,1,52.185030,5.417174,0.000000,0 days 00:26:12
6825,642425887,2023-05-23 10:35:42,1,51.665755,5.054182,0.000000,0 days 00:01:00
6940,642493852,2023-05-23 14:01:23,1,51.804978,5.137354,0.000000,0 days 00:01:00


In [173]:
cleaned_df["speed"] = cleaned_df.sort_values("haversine_distance", ascending=False)["haversine_distance"] / (cleaned_df.sort_values("haversine_distance", ascending=False)["duration"].dt.seconds / 3600)

In [174]:
cleaned_df

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration,speed
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153,NaN,NaT,NaN
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988,0.012223,0 days 00:29:24,0.024946
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365,0.040803,0 days 00:00:59,2.489653
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539,0.280134,0 days 00:01:00,16.808020
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144,0.525633,0 days 00:01:00,31.538005
...,...,...,...,...,...,...,...,...
10376,647742183,2023-06-14 17:13:23,1,52.188936,5.980637,0.764913,0 days 00:01:00,45.894751
10377,647742362,2023-06-14 17:14:23,1,52.193207,5.978572,0.495336,0 days 00:01:00,29.720152
10378,647742547,2023-06-14 17:15:23,1,52.200175,5.979859,0.779757,0 days 00:01:00,46.785417
10379,647742726,2023-06-14 17:16:23,1,52.203792,5.975125,0.515598,0 days 00:01:00,30.935891


## Detecting Trips

In [207]:
def get_final_df(trajectories):

    data = {}
    start_addrs = []
    end_addrs = []
    start_timestamps = []
    end_timestamps = []
    start_longs_and_lats = []
    end_longs_and_lats = []
    distances = []
    durations = []

    for idx, trajectory in enumerate(trajectories):

        start_time = trajectory.df.index[0]
        end_time = trajectory.df.index[-1]

        start_timestamps.append(start_time)
        end_timestamps.append(end_time)

        #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
        durations.append(end_time - start_time)

        start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
        end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])


    #     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
    #     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

        start_longs_and_lats.append(start_xy)
        end_longs_and_lats.append(end_xy)

        distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))

    # data["start_trip_addr"] = start_addrs
    # data["end_trip_addr"] = end_addrs

    data["start_trip_time"] = start_timestamps
    data["end_trip_time"] = end_timestamps

    data["start_gps_point"] = start_longs_and_lats
    data["end_gps_point"] = end_longs_and_lats

    data["trip_distance_in_km"] = distances

    data["trip_duration"] = durations

    final_df = pd.DataFrame(data)
    return final_df

## 1) Using StopSplitter using Diameter and Duration

In [175]:
MAX_DIAMETER = 400
MIN_DURATION = 200 

In [176]:
traj = mpd.Trajectory(cleaned_df[["DateTimeOfPosition", "long", "lat"]], "Allsetra", x='lat', y='long', t='DateTimeOfPosition', crs=4326)

detector = mpd.TrajectoryStopDetector(traj)
stops = detector.get_stop_points(min_duration=timedelta(seconds=MAX_DIAMETER), max_diameter=MIN_DURATION)


In [177]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=MAX_DIAMETER, min_duration=timedelta(seconds=MIN_DURATION))

In [178]:
cleaned_df[:40]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat,haversine_distance,duration,speed
0,631414692,2023-04-01 08:50:30,0,53.389323,5.270153,NaN,NaT,NaN
1,631418242,2023-04-01 09:19:54,1,53.389274,5.269988,0.012223,0 days 00:29:24,0.024946
2,631418385,2023-04-01 09:20:53,1,53.389564,5.270365,0.040803,0 days 00:00:59,2.489653
3,631418504,2023-04-01 09:21:53,1,53.389952,5.274539,0.280134,0 days 00:01:00,16.808020
4,631418653,2023-04-01 09:22:53,1,53.391285,5.282144,0.525633,0 days 00:01:00,31.538005
5,631418767,2023-04-01 09:23:53,1,53.391549,5.285710,0.238279,0 days 00:01:00,14.296737
6,631418902,2023-04-01 09:24:53,1,53.392134,5.283950,0.133610,0 days 00:01:00,8.016588
7,631419027,2023-04-01 09:25:53,1,53.391772,5.284025,0.040559,0 days 00:01:00,2.433521
8,631419170,2023-04-01 09:26:53,0,53.392237,5.284695,0.068171,0 days 00:01:00,4.090260
9,631432987,2023-04-01 11:06:38,1,53.392189,5.284687,0.005364,0 days 01:39:45,0.003226


In [179]:
stops

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-04-01 08:50:30,POINT (5.27015 53.38932),2023-04-01 08:50:30,2023-04-01 09:20:53,Allsetra,1823.0
Allsetra_2023-04-01 09:26:53,POINT (5.28469 53.39219),2023-04-01 09:26:53,2023-04-01 11:07:38,Allsetra,6045.0
Allsetra_2023-04-01 11:11:32,POINT (5.27045 53.38903),2023-04-01 11:11:32,2023-04-01 12:19:03,Allsetra,4051.0
Allsetra_2023-04-01 12:26:22,POINT (5.30668 53.38933),2023-04-01 12:26:22,2023-04-01 12:37:27,Allsetra,665.0
Allsetra_2023-04-01 12:44:19,POINT (5.26995 53.38891),2023-04-01 12:44:19,2023-04-01 14:47:48,Allsetra,7409.0
...,...,...,...,...,...
Allsetra_2023-06-13 11:13:35,POINT (4.84507 51.93224),2023-06-13 11:13:35,2023-06-13 12:26:03,Allsetra,4348.0
Allsetra_2023-06-13 12:53:52,POINT (4.91753 51.74378),2023-06-13 12:53:52,2023-06-13 13:19:15,Allsetra,1523.0
Allsetra_2023-06-13 13:52:28,POINT (5.06723 51.58887),2023-06-13 13:52:28,2023-06-13 14:49:57,Allsetra,3449.0


In [180]:
traj_col

TrajectoryCollection with 342 trajectories

In [209]:
final_stop_splitter_df = get_final_df(traj_col.trajectories)

In [210]:
final_stop_splitter_df

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,2023-04-01 09:21:53,2023-04-01 09:22:53,"(5.274539, 53.389952)","(5.282144, 53.391285)",0.858422,0 days 00:01:00
1,2023-04-01 11:07:38,2023-04-01 11:11:32,"(5.284403, 53.391597)","(5.269931, 53.388944)",1.635806,0 days 00:03:54
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19
3,2023-04-01 12:38:25,2023-04-01 12:44:19,"(5.303019, 53.388009)","(5.269951, 53.388914)",3.678364,0 days 00:05:54
4,2023-04-01 14:47:48,2023-04-01 14:52:01,"(5.272201, 53.389177)","(5.28636, 53.402797)",2.180136,0 days 00:04:13
...,...,...,...,...,...,...
337,2023-06-13 12:26:03,2023-06-13 12:49:16,"(4.846854, 51.931568)","(4.917714, 51.74426)",22.197637,0 days 00:23:13
338,2023-06-13 13:19:15,2023-06-13 13:49:15,"(4.916819, 51.742863)","(5.0676, 51.589485)",23.869822,0 days 00:30:00
339,2023-06-13 14:49:57,2023-06-13 16:16:57,"(5.066662, 51.587983)","(5.973399, 52.203352)",121.672973,0 days 01:27:00
340,2023-06-14 06:43:06,2023-06-14 07:38:21,"(5.974457, 52.203751)","(5.376879, 52.154919)",66.667055,0 days 00:55:15


In [211]:
final_stop_splitter_df.sort_values("trip_duration")

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
228,2023-05-21 16:14:48,2023-05-21 16:15:24,"(5.968685, 52.205315)","(5.967834, 52.205271)",0.094752,0 days 00:00:36
79,2023-04-20 20:53:28,2023-04-20 20:54:11,"(5.97227, 52.203974)","(5.973436, 52.204202)",0.132083,0 days 00:00:43
88,2023-04-23 11:58:12,2023-04-23 11:58:56,"(5.96891, 52.205766)","(5.968092, 52.205389)",0.100058,0 days 00:00:44
183,2023-05-13 08:58:02,2023-05-13 08:58:50,"(5.971067, 52.203252)","(5.973432, 52.203339)",0.263152,0 days 00:00:48
299,2023-06-05 17:34:33,2023-06-05 17:35:29,"(5.971579, 52.204122)","(5.973552, 52.203725)",0.223738,0 days 00:00:56
...,...,...,...,...,...,...
282,2023-05-31 15:03:56,2023-05-31 16:46:34,"(4.437647, 52.211664)","(5.973557, 52.203344)",170.788121,0 days 01:42:38
276,2023-05-31 06:26:58,2023-05-31 08:13:31,"(5.979237, 52.191352)","(4.236802, 51.857055)",197.255797,0 days 01:46:33
194,2023-05-16 07:25:00,2023-05-16 09:18:00,"(5.974655, 52.203694)","(5.06647, 51.586308)",121.931356,0 days 01:53:00
174,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58


## 2) Using ObservationGapSplitter, 3 minutes

In [212]:
splitted_traj = mpd.ObservationGapSplitter(traj).split(gap=timedelta(minutes=3))
dfs = [splitted.df for splitted in splitted_traj.trajectories]

combined_dfs = pd.concat(dfs)

In [199]:
splitted_traj.trajectories

TrajectoryCollection with 321 trajectories

In [214]:
final_gap_splitter_df = get_final_df(splitted_traj.trajectories)

In [232]:
final_gap_splitter_df.sort_values("trip_duration")

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
291,2023-06-06 17:46:55,2023-06-06 17:47:08,"(6.1066, 52.505084)","(6.106607, 52.505129)",0.005036,0 days 00:00:13
112,2023-04-29 09:14:33,2023-04-29 09:14:48,"(6.073165, 52.516777)","(6.073092, 52.516887)",0.014623,0 days 00:00:15
146,2023-05-06 09:00:14,2023-05-06 09:00:31,"(6.137562, 52.154644)","(6.137545, 52.15456)",0.009477,0 days 00:00:17
111,2023-04-29 09:08:04,2023-04-29 09:08:21,"(6.073115, 52.516857)","(6.07319, 52.516729)",0.016427,0 days 00:00:17
316,2023-06-13 12:53:24,2023-06-13 12:53:52,"(4.917784, 51.744315)","(4.917539, 51.74378)",0.065231,0 days 00:00:28
...,...,...,...,...,...,...
313,2023-06-13 06:03:53,2023-06-13 07:53:24,"(5.973009, 52.20328)","(6.002149, 51.185622)",112.587848,0 days 01:49:31
154,2023-05-09 07:21:29,2023-05-09 09:12:37,"(5.973129, 52.203481)","(4.933096, 52.307976)",116.223539,0 days 01:51:08
164,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
183,2023-05-16 07:24:01,2023-05-16 09:18:00,"(5.97284, 52.203622)","(5.06647, 51.586308)",121.759847,0 days 01:53:59


In [223]:
final_gap_splitter_df

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,2023-04-01 09:19:54,2023-04-01 09:26:53,"(5.269988, 53.389274)","(5.284695, 53.392237)",1.667930,0 days 00:06:59
1,2023-04-01 11:06:38,2023-04-01 11:11:32,"(5.284687, 53.392189)","(5.269931, 53.388944)",1.679673,0 days 00:04:54
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19
3,2023-04-01 12:37:27,2023-04-01 12:44:19,"(5.306676, 53.389332)","(5.269951, 53.388914)",4.083902,0 days 00:06:52
4,2023-04-01 14:46:48,2023-04-01 14:52:01,"(5.269951, 53.388905)","(5.28636, 53.402797)",2.386449,0 days 00:05:13
...,...,...,...,...,...,...
316,2023-06-13 12:53:24,2023-06-13 12:53:52,"(4.917784, 51.744315)","(4.917539, 51.74378)",0.065231,0 days 00:00:28
317,2023-06-13 13:17:16,2023-06-13 13:52:28,"(4.917536, 51.743775)","(5.067226, 51.588894)",23.903979,0 days 00:35:12
318,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59
319,2023-06-14 06:42:08,2023-06-14 07:38:21,"(5.973082, 52.203594)","(5.376879, 52.154919)",66.513258,0 days 00:56:13


In [225]:
df[df["DateTimeOfPosition"] == "2023-05-31 06:25:58"]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
8104,644138054,2023-05-31 06:25:58,1,52.193825,5.976286


In [233]:
df[8105:8175]

,UnitpositionID,DateTimeOfPosition,IgnitionOn,long,lat
8105,644138198,2023-05-31 06:26:58,1,52.191352,5.979237
8106,644138360,2023-05-31 06:27:58,1,52.183422,5.982220
8107,644138502,2023-05-31 06:28:58,1,52.179351,5.974677
8108,644138667,2023-05-31 06:29:58,1,52.178980,5.962479
8109,644138842,2023-05-31 06:30:58,1,52.172314,5.962010
...,...,...,...,...,...
8170,644154060,2023-05-31 07:31:59,1,52.039414,4.702675
8171,644154367,2023-05-31 07:32:59,1,52.030925,4.685066
8172,644154659,2023-05-31 07:33:59,1,52.026359,4.669961
8173,644154946,2023-05-31 07:34:59,1,52.020984,4.646997
